# Introduction to CUDA and PyCUDA

Install the required module `pycuda`, the `rasterio` is for geotiff file operation. We will use it for raster data operations.

#### Import the required `pycuda` libraries

In [1]:
import pycuda.autoinit
import pycuda.driver as cuda
import numpy as np
from pycuda.compiler import SourceModule

In [2]:
# Define CUDA kernel
mod = SourceModule("""
__global__ void add_vectors(float *a, float *b, float *c, int N) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;

    if (idx < N) {
        c[idx] = a[idx] + b[idx];
    }
}
""")

# Get the kernel function
add_vectors = mod.get_function("add_vectors")


C:\Users\Tess\AppData\Local\Temp\ipykernel_82428\2039429436.py:2: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu

  mod = SourceModule("""


In [4]:
# Create input data
N = 1024
a = np.random.randn(N).astype(np.float32)
b = np.random.randn(N).astype(np.float32)
c = np.empty_like(a)
a, b, c

(array([ 0.18346167,  0.9063867 , -0.7365927 , ...,  0.1067035 ,
        -1.9502953 , -0.5399432 ], shape=(1024,), dtype=float32),
 array([-1.0885758 ,  0.11922219, -0.78583676, ...,  0.93014514,
         0.0380047 ,  0.07656984], shape=(1024,), dtype=float32),
 array([0., 0., 0., ..., 0., 0., 0.], shape=(1024,), dtype=float32))

In [5]:
# Allocate GPU memory
a_gpu = cuda.mem_alloc(a.nbytes)
b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(c.nbytes)

# Copy data from host (CPU) to device (GPU)
cuda.memcpy_htod(a_gpu, a)
cuda.memcpy_htod(b_gpu, b)


# Define grid and block size
block_size = 256
grid_size = (N + block_size - 1) // block_size

# Launch the kernel
add_vectors(a_gpu, b_gpu, c_gpu, np.int32(N), block=(block_size, 1, 1), grid=(grid_size, 1, 1))

# Copy result back from device (GPU) to host (CPU)
cuda.memcpy_dtoh(c, c_gpu)

In [6]:
c_gpu

In [7]:
c.shape

(1024,)

In [8]:

# Verify the result
print("First 5 results:")
print("GPU result:", c[:5])
print("Expected:", (a + b)[:5])

# Free GPU memory (optional, PyCUDA auto-manages it)
a_gpu.free()
b_gpu.free()
c_gpu.free()


First 5 results:
GPU result: [-0.9051142   1.0256089  -1.5224295  -0.8016787  -0.13093054]
Expected: [-0.9051142   1.0256089  -1.5224295  -0.8016787  -0.13093054]


# Exercises

1. Write a cuda kernel to find the elementwise square of a matrix
2. Write a cuda kernel to multiply three matrices: